In [1]:
from datetime import datetime
from netCDF4 import Dataset as ds
import numpy as np
import dateutil as du 
import parser
import pytz
import dateutil.parser 
import pandas as pd
from pandas import DataFrame
import csv
#import glob, os
from os.path import expanduser
from pathlib import Path
import xarray as xr 
import datetime as dt

In [2]:
### Load .CSV and 
### subset bleaching events (conditional, e.g. exclude NaN)
### Filter only CodeID, site names and country" 
df = pd.read_csv('CBdbV2.csv',encoding='latin-1', low_memory=False)
df = df[['RECORD_ID','COUNTRY','LAT_corrected','LON_corrected','SEVERITY_CODE','DATE','MONTH','YEAR']]
df = pd.DataFrame(df)
#### Convert index into column
#df.reset_index(level=0, inplace=True)
df['ITEM_ID'] = df.index.astype(str).str.zfill(7)   # add '0s' at the beginning of the number and convert as string 
df.rename(columns={'DATE':'DAY'}, inplace = True) #use "implace = True" to rename the existing DataFrame
df = df[pd.to_numeric(df['DAY'], errors='coerce').notnull()]  # Drop rows that do not have as.numeric DATE (DAY)
df['DAY'] = df['DAY'].apply(lambda x: int(float(x)))
df['DAY'] = df['DAY'].apply('{0:0>2}'.format)
df['MONTH'] = df['MONTH'].apply(lambda x: int(float(x)))
df['MONTH'] = df['MONTH'].apply('{0:0>2}'.format)
df['full_date'] = df.apply(lambda x: '%s-%s-%s' % (x['YEAR'], int(float(x['MONTH'])), x['DAY']), axis=1) # create a column in datetime format; %s - String (or any object with a string representation, like numbers), %d - Integers %f - Floating point numbers
df['full_date'] = df['full_date'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d")) # use datetime funtion to convert string 'full_date' to datetime  
df['JD'] = df['full_date'].apply(lambda x: x.timetuple().tm_yday)
### Re-order by column starting by 'ID' and renaming LAT and LON
df = df.reindex(columns=['ITEM_ID','COUNTRY','LAT_corrected','LON_corrected','SEVERITY_CODE','DAY','MONTH','YEAR','full_date','JD'])
df.rename(columns={'LAT_corrected': 'lat', 'LON_corrected': 'lon'}, inplace=True) # use "implace = True" to rename the existing DataFrame (rather than creating a copy)
df = df.dropna(axis=0, how='any')  # Drop row with NaN if any

In [3]:
df

,ITEM_ID,COUNTRY,lat,lon,SEVERITY_CODE,DAY,MONTH,YEAR,full_date,JD
358,0000358,United States,28.164800,-94.298300,1,11,10,1993,1993-10-11,284
384,0000384,United States,28.164800,-94.298300,1,14,05,1994,1994-05-14,134
390,0000390,United States,28.164800,-94.298300,0,25,09,1994,1994-09-25,268
477,0000477,United States,28.164800,-94.298300,1,14,05,1995,1995-05-14,134
495,0000495,United States,28.164800,-94.298300,1,08,10,1995,1995-10-08,281
...,...,...,...,...,...,...,...,...,...,...
37768,0037768,Iran,26.661884,55.915603,0,20,12,2017,2017-12-20,354
37769,0037769,Jamaica,18.099944,-77.999472,0,20,12,2017,2017-12-20,354
37770,0037770,Australia,-17.290650,146.478083,0,21,12,2017,2017-12-21,355
37771,0037771,Iran,26.826644,56.319456,0,21,12,2017,2017-12-21,355


In [4]:
# Function to slice the filename with .nc extention, this will be used to open the files in directory
def file_info(ncfilename):
    with ds(ncfilename,'r') as ncdat:
        #string = str(ncfilename)
        #filename = string[-16:] 
        array = xr.open_dataset(ncfilename) # open_dataset works fine with either full path or filename.nc 
        return array  
def array_sel(array):
    dsloc = array.sel(lon=lon,lat=lat,time=time,method='nearest') 
    dhw = dsloc['degree_heating_week']
    return dhw 

In [5]:
home = Path(expanduser("~"))
x = []
for row in df.itertuples():
        lat = row.lat,
        lon = row.lon
        time = row.full_date
        path = home / Path("D:/PGonzalez/DHW/DHW_monmax/dhw_1985-2017_runmax30.nc") 
        array = file_info(path)
        dhw_val = array_sel(array)
        x.append(dhw_val)
dhw = [item[0].values for item in x]
df['DHWrunmax30'] = dhw
df.to_csv('dhw_dateofreportrunmax30.csv')

In [ ]:
x

In [ ]:
filename = 'dhw_1985-2017_monmax.nc'   
array = xr.open_dataset(filename)  
dsloc = array.sel(lon=-109.9350,lat=24.2210,time=20171115
,method='nearest') 
dhw = dsloc['degree_heating_week']


In [ ]:
dhw